In [3]:
%run "./variables_globales.ipynb"

# Plantillas de codigo
Funciones para hacer las plantillas

In [ ]:
def crear_funcion(instruccion):
    """
    Crea el template de la estructura de una función

    Parametros
    ----------
    instrucion: str
        La intruccion de voz en texto.

    Regresa
    ---------
    output: str
        Codigo generado
    recomendacion: str
        Una sugerencia o fallo
    """

    global indentacion
    global recomendacion

    # guarda los avisos o recomendaciones que el programa te hace
    recomendacion = ''

    # guarda la línea de código
    output = ''
    
    # pivote que ayuda a definir el nombre de una función
    before_keyword, keyword, after_keyword = instruccion.partition('nombre')

    # verifica que haya o esté escrita la frase "nombre"
    if len(after_keyword) == 0:
        recomendacion = f'¡No me dijiste el nombre de la función!'

    # de otro modo, si tiene nombre la función
    else:

        # obtenemos el nombre de la función por el usuario
        name_func = after_keyword.split(' ')[1]

        # verificamos si no desea poner parametros                                
        if instruccion.strip().split(' ')[-1] == name_func:
            parametros = ''

        # de otro modo, si desea una función con parámetros
        else:
            before_keyword, keyword, after_keyword = instruccion.partition('parametros')

            # verifica que si exista el nombre de los parámetros
            if len(after_keyword) == 0:
                parametros = ''
                recomendacion = f'¡No me dijiste el nombre de los parámetros!'

            # escribe como parámetros todo lo que está después de "parámetros"
            else:
                parametros = after_keyword.split(' ')[1:]
                parametros = ', '.join(parametros)

        # indenta aunque marque que detecte que no le dije parámetros
        if not recomendacion or recomendacion == '¡No me dijiste el nombre de los parámetros!':
            indentacion += 1

        # concatenación del nombre y parámetros de la función
        output = f'def {name_func}({parametros}):\n' + '\t' * indentacion
    return output

#### Prueba para la funcion "crear_funcion"

In [ ]:
crear_funcion('definir función con nombre magica y parametros x y z')[0]

In [7]:
def crear_condicional(transcript):
    ''' 
    Toma transcript y condicional (if,elif,while) y devuelve código
    El transcript ya está preprocesado para comenzar con la palabra "mientras"

    Test
    >>> mientras variable india sea menor igual a numero seis => while (i<=6):
    '''    
    global bloque

    keyword_mapeo={'mientras':'while','si':'if','no si':'elif'}
    antes_keyword,keyword,desp_keyword=encontrar_palabras(transcript,keyword_mapeo.keys())
    cadena=keyword_mapeo[keyword]
    bloque = keyword

    # Primera división
    condicional_mapeo={'menor estricto':'<','menor o igual':'<=','igual':'==','diferente':'!='
    ,'mayor estricto':'>','mayor o igual':'>='}
    cjto_condicional=condicional_mapeo.keys()
    antes_condicional,palabra_condicional,despues_condicional=encontrar_palabras(transcript,cjto_condicional) 
    
    
    # Buscar en antes condicional la lista de variables
    

    a_var,var,d_var=encontrar_palabras(antes_condicional,['variable'])
    nombre_var=d_var.split(' ')[0]

    if diccionario_fonetico.get(nombre_var,False):
        nombre_var=diccionario_fonetico[nombre_var]


    cadena+=' '+nombre_var+' ' +condicional_mapeo[palabra_condicional]

    # Buscar en despues_condicional el número 

    valor=despues_condicional.split(' ')[-1]

    if dict_numeros.get(valor,False):
        valor=str(dict_numeros[valor])

    global indentacion
    indentacion+=1

    return cadena+' '+ valor+':\n'+ '\t'* indentacion+'|'

#### Prueba para la funcion "crear_condicional"

In [8]:
indentacion = 0
test_frases_condicional = [
    'si variable india es menor o igual a numero seis',
    'mientras variable alpha es menor igual a numero dos'
    ]

for ai in test_frases_condicional:    
    print(crear_condicional(ai))    

('if i <= 6:\n\t|', '')
('while alpha == 2:\n\t\t|', '')


In [9]:
def crear_cadena(transcript):
    cjto_palabras=["'",'"']
    try:
        inicio,final=list(re.finditer(r"cadena (.+) cadena",transcript))[0].span()
    except:
        return ''
    antes_palabra=transcript[:inicio].strip()
    despues_palabra=transcript[final:].strip()
    palabra=list(re.finditer(r"cadena (.+) cadena",transcript))[0].group(1)
    return antes_palabra,palabra,despues_palabra


#### Prueba para la funcion "crear_funcion"

In [10]:
test_str_cadena=[
    'ejecuta print con argumentos variable India producto cadena guión cadena',
    'ejecuta print con argumentos cadena Hola mundo cadea'
    ]

for ti in test_str_cadena:
    print(crear_cadena(ti))

('ejecuta print con argumentos variable India producto', 'guión', '')



In [13]:
def crear_var_existente(transcript):
    try:
        antes_var,var,desp_var=encontrar_palabras(transcript,['variable'])
    except:
        return '' 
    return desp_var.split(' ')[0]

#### Test crear_var_existente

In [14]:
test_str_cadena=[
    'ejecuta print con argumentos variable India producto cadena guión cadena',
    'ejecuta print con argumentos cadena Hola mundo cadea'
    ]

for ti in test_str_cadena:
    print(crear_var_existente(ti))

India



In [11]:
# TODO: Hay que ver:
    # Si es otra operación hay que llamar la función recursivamente en cada pedazo
    # 1. si es cadena
    # 2. si es otra operación. Para esto, hay que regresar error o algo así cuando no se encuentre
def crear_operacion(transcript):
    '''
    Debe poder tomar una secuencia e inferir si se están haciendo operaciones
    Al llamar una función entre cada coma ver si se están haciendo operaciones

    ej
    >> crear_operacion('variable India producto cadena guión cadena') 
    >> i*'-'
    '''

    # Separa en operadores comunes
    dict_operaciones={'producto':'*','suma':'+','menos':'-'
    ,'concatena':'+','entre':'/'} # si esto se lematiza puedes agarrar todas las frases de la forma suma, sumar, etc.

    try:
        antes_op,op,desp_op=encontrar_palabras(transcript,dict_operaciones.keys())
    except:
        return '' 

    # Buscamos la información en la cadena detrás del operador
    var_izq=crear_var_existente(antes_op)
    try:
        cadena_izq=f'"{crear_cadena(antes_op)[1]}'
    except:
        cadena_izq=''
        
        
    # Buscamos la información en la cadena después del operador
    var_der=crear_var_existente(desp_op)
    try:
        cadena_der=f'"{crear_cadena(desp_op)[1]}"'
    except:
        cadena_der=''
                
    return f'{var_izq}{cadena_izq} {dict_operaciones[op]} {var_der}{cadena_der}'

#### Test crear_operacion

In [15]:
test_str_operacion=[
    'variable India producto cadena guión cadena',
    'variable India menos variable Francis','20 mil entre 3' #checar tercer caso, como agarrar valor solamente
    ] 

for ti in test_str_operacion:
    print(crear_operacion(ti))

India * "guión"
India - Francis
 / 


In [ ]:
def crear_llamada(transcript):
    """Template para la llamada de una funcion"""
    antes_ej,ej,desp_ej=encontrar_palabras(transcript,['ejecuta'])
    funcion_nombre=desp_ej[0]
    # Aquí tal vez valdría la pena tener un registro de las funciones previamente definidas para
    # poder buscar en un directorio con Jaccard y no aproximar

    antes_arg,keyword,desp_arg=encontrar_palabras(desp_ej,['argumentos','parametros'])

    argumentos=desp_arg.split('coma')

    for arg in argumentos:
        print(arg,crear_operacion(arg))
    cadena=f'{funcion_nombre}()'

    return argumentos

In [ ]:
test_str_ej=[
    'ejecuta print con argumentos variable India producto cadena guión cadena coma cadena hola cadena'
]
for ti in test_str_ej:
    print(crear_llamada(ti))

In [16]:
def crear_variable(instruccion):
    """
    Estructura:
    definir variable con nombre [nombre_variable] igual a /*objeto_basico* valor/ 

    Parametros
    ----------
    instrucion: str
        La intruccion de voz en texto.

    Regresa
    ---------
    output: str
        Codigo generado
    recomendacion: str
        Una sugerencia o fallo

    Testing
    -------
    >>> definir variable con nombre india igual a número uno
    >>> Definir variable con nombre i igual a numero 1 (int)
    >>> Definir variable con nombre i igual a numero 3.5 (float) 
    >>> Definir variable con nombre i igual a cadena hola (string) 
    >>> Definir variable con nombre i igual a lista/dic (string)   
    """     
    global indentacion

    # pivote que ayuda a definir el nombre de la variable
    before_keyword, keyword, after_keyword = instruccion.partition('nombre')
    after_keyword_list = after_keyword.strip().split(' ')
    name_variable = after_keyword_list[0]

    
    # Como sabemos que despues del nombre va seguido de "igual a"
    tipo_dato = after_keyword_list[3]        
    #print(after_keyword_list[4:]) -> lista
    valor = tipos_datos[tipo_dato](after_keyword_list[4:][0])        
    # Verificamos si es una palabra fonetica
    if diccionario_fonetico.get(name_variable,False):
        name_variable=diccionario_fonetico[name_variable] 

    codigo_generado = f'{name_variable} = {valor}\n'+ '\t' * indentacion + '|'
    return codigo_generado, ''   

####  Prueba para la funcion "crear_variable"

In [17]:
crear_variable('definir variable con nombre india igual a numero cuatro')

('i = 4\n\t\t|', '')

### Funciones pendientes

In [ ]:
def crear_clase():
    global indentacion
    # ....
    
    indentacion += 1
    pass    

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=17740a58-e960-4de3-bb66-1575286930f1' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>